<a href="https://colab.research.google.com/github/gohmrmts/RoK_OCR_TOTALPOWER/blob/main/OCR_TOTAL_POWER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !apt install tesseract-ocr libtesseract-dev tesseract-ocr-jpn
 !pip install pyocr
 !pip install Pillow

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libtesseract-dev is already the newest version (4.00~git2288-10f4998a-2).
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
tesseract-ocr-jpn is already the newest version (4.00~git24-0e00fe6-1.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# -*- coding: utf-8 -*-
import sys
from PIL import Image, ImageOps
import os

# 正規表現
import re

# トリミング前の画像の格納先
ORIGINAL_FILE_DIR = "/content/original/"
# トリミング後の画像の格納先
TRIMMED_FILE_DIR = "/content/trimed/"


if __name__ == '__main__':
  # もしトリミング後の画像の格納先が存在しなければ作る
  if os.path.isdir(TRIMMED_FILE_DIR) == False:
    os.makedirs(TRIMMED_FILE_DIR)
  if os.path.isdir(ORIGINAL_FILE_DIR) == False:
    os.makedirs(ORIGINAL_FILE_DIR)

**画像をoriginalにアップロード**

In [ ]:
# -*- coding: utf-8 -*-
import sys
from PIL import Image, ImageOps
import os

# 正規表現
import re


# 画像パスと、左上座標、右下座標を指定して、トリミングされたimageオブジェクトを返す。
def trim(path, left, top, right, bottom):
  im = Image.open(path)
  im_trimmed = im.crop((left,top,right,bottom))
  im_trimmed = ImageOps.invert(im_trimmed.convert('RGB'))
  return im_trimmed


if __name__ == '__main__':
  # for IPad
  # トリミングする左上の座標
  left, top = 1650, 550
  # トリミングする右上の座標
  right, bottom = 1850, 1400

  # 画像ファイル名を取得
  files = os.listdir(ORIGINAL_FILE_DIR)
  # 特定の拡張子のファイルだけを採用。実際に加工するファイルの拡張子に合わせる
  files = [name for name in files if name.split(".")[-1] in ["png", "PNG", "jpg", "JPG"]]

  for val in files:
    # オリジナル画像へのパス
    path = ORIGINAL_FILE_DIR + val
    # トリミングされたimageオブジェクトを取得
    im_trimmed = trim(path, left, top, right, bottom)
    # グレースケール
    # im_trimmed = im_trimmed.convert("L")
    # モノクローム
    # im_trimmed = im_trimmed.convert("1")

    # 画像を読みやすいように加工。
    size=im_trimmed.size
    img2=Image.new('RGB',size)
 
    border=110
 
    for x in range(size[0]):
        for y in range(size[1]):
            r,g,b=im_trimmed.getpixel((x,y))
            if r > border or g > border or b > border:
                r = 255
                g = 255
                b = 255
            im_trimmed.putpixel((x,y),(r,g,b))
 

    # トリミング後のディレクトリに保存。ファイル名の頭に"cut_"をつけている
    im_trimmed.save(TRIMMED_FILE_DIR+"cut_"+val, quality=95) #qualityは95より大きい値は推奨されていないらしい

In [ ]:
# OCR関連
from PIL import ImageFile 
ImageFile.LOAD_TRUNCATED_IMAGES = True

from PIL import Image

import pyocr

# spreadsheet関連 認証のためのコード
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# pyocr関連
tools = pyocr.get_available_tools()
tools
tool = tools[0]
tool.get_name()

'Tesseract (sh)'

In [ ]:
# 現在日時
import datetime

dt_now = datetime.datetime.now()
print(dt_now)

2022-08-30 18:32:26.364129


In [ ]:
## スプレッドシートの作成
filename = "RoK_kingdom_power_" + str(dt_now.year) + "/" + str(dt_now.month) + "/" + str(dt_now.day) + "/" + str(dt_now.hour) + ":" + str(dt_now.minute)
ss = gc.create(filename)
print(ss.title)

# シートを特定する（シート名で特定）
st = ss.worksheet("シート1")

RoK_kingdom_power_2022/8/30/18:32


In [ ]:
# 画像ファイル名を取得
files = os.listdir(TRIMMED_FILE_DIR)
# 特定の拡張子のファイルだけを採用。実際に加工するファイルの拡張子に合わせる
files = [name for name in files if name.split(".")[-1] in ["png", "PNG", "jpg", "JPG"]]
# ソート
files.sort()

In [ ]:
# OCR
strlist = []
for filename in files:
  img = Image.open(TRIMMED_FILE_DIR + filename)
  txt = tool.image_to_string(
      img,
      lang = "jpn",
      builder = pyocr.builders.TextBuilder(tesseract_layout=6)
      # builder = pyocr.builders.DigitBuilder()
      # builder = pyocr.builders.LineBoxBuilder(tesseract_layout=6)
      )
  strlist += txt.split("\n")
  print(filename)

cut_IMG_1757.PNG
cut_IMG_1758.PNG
cut_IMG_1759.PNG
cut_IMG_1760.PNG
cut_IMG_1761.PNG
cut_IMG_1762.PNG
cut_IMG_1763.PNG
cut_IMG_1764.PNG
cut_IMG_1765.PNG
cut_IMG_1766.PNG
cut_IMG_1767.PNG
cut_IMG_1768.PNG
cut_IMG_1769.PNG
cut_IMG_1770.PNG
cut_IMG_1771.PNG
cut_IMG_1772.PNG
cut_IMG_1773.PNG
cut_IMG_1774.PNG
cut_IMG_1775.PNG
cut_IMG_1776.PNG
cut_IMG_1777.PNG
cut_IMG_1778.PNG
cut_IMG_1779.PNG
cut_IMG_1780.PNG
cut_IMG_1781.PNG
cut_IMG_1782.PNG
cut_IMG_1783.PNG
cut_IMG_1784.PNG
cut_IMG_1785.PNG
cut_IMG_1786.PNG
cut_IMG_1787.PNG
cut_IMG_1788.PNG
cut_IMG_1789.PNG
cut_IMG_1790.PNG
cut_IMG_1791.PNG
cut_IMG_1792.PNG
cut_IMG_1793.PNG
cut_IMG_1794.PNG
cut_IMG_1795.PNG
cut_IMG_1796.PNG
cut_IMG_1797.PNG
cut_IMG_1798.PNG
cut_IMG_1799.PNG
cut_IMG_1800.PNG
cut_IMG_1801.PNG
cut_IMG_1802.PNG
cut_IMG_1803.PNG
cut_IMG_1804.PNG
cut_IMG_1805.PNG
cut_IMG_1806.PNG


In [ ]:
print(len(strlist))
strlist

300


['163,775,505',
 '128,862,602',
 '117,608,554',
 '110,679,622',
 '105,445,161',
 '104,381,833',
 '104,342,152',
 '104,005,980',
 '102,252,833',
 '101,730,049',
 '100,403,238',
 '100,196,554',
 '100,096,334',
 '98,.336,754',
 '97.648,213',
 '95,956,519',
 '95,436,465',
 '94,880,924',
 '94,663,590',
 '93,065,621',
 '92,765,910',
 '92,600,191',
 '91,973,122',
 '91,533,745',
 '91,463,465',
 '90,690,788',
 '89,965,711',
 '89.837.036',
 '89.177.910',
 '88,750,639',
 '88,727,293',
 '87,924,253',
 '87,833,127',
 '87,664,676',
 '87,524,743',
 '87,301,539',
 '86,149,106',
 '85,843,770',
 '85,607,438',
 '85,254,652',
 '85,095,354',
 '85,093,855',
 '84,830,959',
 '84,783,069',
 '84,194,069',
 '83,664,523',
 '83,487,351',
 '83,082,998',
 '82,708,743',
 '82,545,834',
 '82,505,625',
 '82,340,212',
 '81,582,450',
 '81,160,013',
 '81,082,241',
 '81,065,590',
 '80,998,987',
 '80,925,890',
 '80,796,351',
 '80,578,497',
 '80,169,504',
 '80,096,907',
 '79.593,364',
 '79.593,157',
 '79,449,980',
 '79,022,83

In [ ]:
# データ整形
arr = []
for i in range(len(strlist)):
  if not strlist[i]:
    pass
  else :
    arr.append(strlist[i])

strlist.clear()

pattern = '\d'
for i in range(len(arr)) :
  if(re.match(pattern, arr[i][0])):
    arr[i] = arr[i].replace(",", "")
    arr[i] = arr[i].replace(".", "")
    arr[i] = arr[i].replace(" ", "")
    strlist.append(arr[i])


In [ ]:
import copy
strlist_row = copy.copy(strlist)
strlist

['163775505',
 '128862602',
 '117608554',
 '110679622',
 '105445161',
 '104381833',
 '104342152',
 '104005980',
 '102252833',
 '101730049',
 '100403238',
 '100196554',
 '100096334',
 '98336754',
 '97648213',
 '95956519',
 '95436465',
 '94880924',
 '94663590',
 '93065621',
 '92765910',
 '92600191',
 '91973122',
 '91533745',
 '91463465',
 '90690788',
 '89965711',
 '89837036',
 '89177910',
 '88750639',
 '88727293',
 '87924253',
 '87833127',
 '87664676',
 '87524743',
 '87301539',
 '86149106',
 '85843770',
 '85607438',
 '85254652',
 '85095354',
 '85093855',
 '84830959',
 '84783069',
 '84194069',
 '83664523',
 '83487351',
 '83082998',
 '82708743',
 '82545834',
 '82505625',
 '82340212',
 '81582450',
 '81160013',
 '81082241',
 '81065590',
 '80998987',
 '80925890',
 '80796351',
 '80578497',
 '80169504',
 '80096907',
 '79593364',
 '79593157',
 '79449980',
 '79022836',
 '78990728',
 '78626097',
 '78579859',
 '78436982',
 '77713039',
 '77644093',
 '77622719',
 '77435232',
 '77434486',
 '77413284',

In [ ]:
print("---7の誤検出の修正---")
for k in range(5):
  err = []
  for i in range(2, len(strlist) - 1):
    if(int(strlist[i - 1]) >= int(strlist[i]) and int(strlist[i]) >= int(strlist[i + 1])):
      pass
    else:
      err.append(i)

  print(err)

  for i in range(len(err)):
    if i % 2 == 0:
      print(strlist[err[i]])
      if(strlist[err[i]][0] == "1"):
        if(strlist[err[i] - 1][0] == "7" or strlist[err[i] + 1][0] == "7"):
          print(err[i], " : ", strlist[err[i]], "->", "7" + strlist[err[i]][1:])
          strlist[err[i]] = "7" + strlist[err[i]][1:]
          

print("\n---そのほかの修正---")
if err:
  for i in range(len(err)):
    if i % 2 == 0:
      avg = int((int(strlist[err[i] - 1]) + int(strlist[err[i] + 1])) / 2)
      print(err[i], " : ", strlist[err[i]], "->", avg)
      strlist[err[i]] = str(avg)

---7の誤検出の修正---
[]
[]
[]
[]
[]

---そのほかの修正---


In [ ]:
# 300になることを確認
len(strlist)

300

In [ ]:
# スプレッドシート書き込みのためにListに格納
data = []
for i in range(len(strlist)):
  data.append([i + 1, strlist_row[i].strip(), strlist[i].strip()])

In [ ]:
# 総戦力の計算
total = 0
total_row = 0
for i in range(len(strlist)):
  total += int(strlist[i])
  total_row += int(strlist_row[i])

data = [["TOTAL", total_row, total]] + data

In [ ]:
for i in range(len(data)) :
  if data[i][1] != data[i][2]:
    data[i] += ["fixed"]

In [ ]:
data

[['TOTAL', 19105606228, 19105606228],
 [1, '163775505', '163775505'],
 [2, '128862602', '128862602'],
 [3, '117608554', '117608554'],
 [4, '110679622', '110679622'],
 [5, '105445161', '105445161'],
 [6, '104381833', '104381833'],
 [7, '104342152', '104342152'],
 [8, '104005980', '104005980'],
 [9, '102252833', '102252833'],
 [10, '101730049', '101730049'],
 [11, '100403238', '100403238'],
 [12, '100196554', '100196554'],
 [13, '100096334', '100096334'],
 [14, '98336754', '98336754'],
 [15, '97648213', '97648213'],
 [16, '95956519', '95956519'],
 [17, '95436465', '95436465'],
 [18, '94880924', '94880924'],
 [19, '94663590', '94663590'],
 [20, '93065621', '93065621'],
 [21, '92765910', '92765910'],
 [22, '92600191', '92600191'],
 [23, '91973122', '91973122'],
 [24, '91533745', '91533745'],
 [25, '91463465', '91463465'],
 [26, '90690788', '90690788'],
 [27, '89965711', '89965711'],
 [28, '89837036', '89837036'],
 [29, '89177910', '89177910'],
 [30, '88750639', '88750639'],
 [31, '88727293

In [ ]:
# 2次元配列を一括で書き込み

ss.values_append("シート1", {"valueInputOption": "USER_ENTERED"}, {"values": data})

{'spreadsheetId': '1NjJ659eCJ1SNHtHLGDXwypo4zvC5xrrLMtTZATKdiRg',
 'tableRange': "'シート1'!A1:D301",
 'updates': {'spreadsheetId': '1NjJ659eCJ1SNHtHLGDXwypo4zvC5xrrLMtTZATKdiRg',
  'updatedRange': "'シート1'!A302:C602",
  'updatedRows': 301,
  'updatedColumns': 3,
  'updatedCells': 903}}